In [ ]:
import os
from theoffice.utils.io import update_env_secrets
update_env_secrets(os.path.join(os.getcwd(), "..", ".secrets", "secrets.yaml"))
GOOGLE_API_KEY = os.environ.get('GEMINI_API_KEY')

# Loading PDFs

Loader has been added to utils

In [ ]:
from theoffice.utils.io import read_pdf_files

pdfs = read_pdf_files(os.path.join(os.getcwd(), '..', 'binaries', 'class_102'))
pdfs

Each element of the list is a `langchain` `Document` object

In [ ]:
pdfs[0]

Documents have certain attributes and methods we can check

In [ ]:
print(*[att for att in dir(pdfs[0]) if not att.startswith("_")])

Metadata is one of these attributes

In [ ]:
pdfs[0].metadata.keys()

In [ ]:
pdfs[0].metadata['total_pages']

Document content can be accesssed in `page_content` attribute

In [ ]:
pdfs[0].page_content

In [ ]:
len(pdfs[0].page_content)

## Chunking the documents

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

In [ ]:
chunks = splitter.split_documents(pdfs)

Chunks are documents too

In [ ]:
chunks[0]

In [ ]:
chunks[0].page_content

In [ ]:
len(chunks[0].page_content)

In [ ]:
for chunk in chunks:
    print(chunk)
    print("-" * 100)

# Embedding Models

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS  # Facebook AI Similarity Search

In [ ]:
embeddings_model = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [ ]:
vector_store = FAISS.from_documents(documents=chunks, embedding=embeddings_model)
vector_store

In [ ]:
chunks[1].lc_id

In [ ]:
vector_store.get_by_ids([0])

In [ ]:
vector_store.similarity_search("viagem", k=1)

In [ ]:
faiss_index = vector_store.index  
faiss_index

In [ ]:
# Number of vectors
print(faiss_index.ntotal), print(len(chunks))

In [ ]:
# Dump all vectors as numpy array
import numpy as np

xb = faiss_index.reconstruct_n(0, faiss_index.ntotal)  # reconstruct all vectors
print(np.array(xb).shape)   # (num_chunks, embedding_dim)

In [ ]:
vec0 = faiss_index.reconstruct(0)
print(vec0.shape)
print(vec0[:10])  # first 10 dimensions

In [ ]:
for i, (doc_id, doc) in enumerate(vector_store.docstore._dict.items()):
    print(i, doc_id)
    vector = faiss_index.reconstruct(i)
    print("Vector sample:", vector[:5])
    print("Doc:")
    print("```")
    print(doc.page_content[:50])
    print("```")
    print("...")
    print("-" * 100)


In [ ]:
retriever = vector_store.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.3, 'k': 4}
)

# Agent Chain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
from theoffice.output_structures.class_101 import TriagemOut
from langchain_core.messages import SystemMessage, HumanMessage
from typing import Dict

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [ ]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()

In [ ]:
prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
]
)


# creating a chain (langchain), so that later one can pass a list of Documents to a LLM chain model
document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [ ]:
def perguntar_politica_RAG(pergunta: str, retriever):
    """Returns a dict"""
    docs_relacionados = retriever.invoke(pergunta)
    if not docs_relacionados:
        # empty results from retriever
        return {
            "resposta": "nao sei",
            "citacoes": [],
            "contexto_encontrado": False
        }
    else:
        return docs_relacionados
    

In [ ]:
def processar_pergunta_com_RAG(pergunta: str, retriever, document_chain):
    docs_relacionados = perguntar_politica_RAG(pergunta, retriever)
    # print("RAG output", docs_relacionados)
    
    answer = document_chain.invoke(
        {
            "input": pergunta,
            "context": docs_relacionados
        }
    )

    txt = (answer or "").strip()
    # print('chain answer', txt)
    
    if txt.rstrip(".!?") == "Não sei":
        ## LLM model answer
        agent_output = {
                "resposta": "nao sei",
                "citacoes": [],
                "contexto_encontrado": False
            }
    else:
        agent_output = {
                "resposta": txt,
                "citacoes": [docs_relacionados],
                "contexto_encontrado": True
            }
    
    return agent_output

In [ ]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

for msg_teste in testes:
    resposta = processar_pergunta_com_RAG(msg_teste, retriever, document_chain)
    print(f"Pergunta: {msg_teste}")
    print(f"Resposta: {resposta['resposta']}")
    print(f"Contexto Encontrado?: {resposta['contexto_encontrado']}")
    print(f"Citacoes: {resposta['citacoes']}")
    print("-" * 100)